
# Water Balance & Runoff Calculator (OpenET + GRIDMET + USGS)

This notebook computes watershed-scale annual **runoff depth** by comparing **precipitation (P)** from GRIDMET and **evapotranspiration (ET)** from OpenET (P − ET), and then relates those values to **observed discharge** from a USGS gage to evaluate agreement.

**You will:**
1. Authenticate and initialize **Google Earth Engine (GEE)**.
2. Load **OpenET** and **GRIDMET**(Other Precip products can be utilized such as PRISM or DAYMET V4 available in GEE) data; compute annual sums and polygon means for a watershed (Watershed data can be pullued from (StreamStats: https://streamstats.usgs.gov/ss/)) or utilizing the code Basin_Shapefile_05470500.ipynb in in GitHub repository: https://github.com/miguel-source/Water-Balance-Analysis-Tool
3. Export optional rasters to Google Drive.
4. Load **USGS** discharge data, compute annual runoff depth (mm) using watershed area. The data can be extracted from NLDI using the hydrofunction python. Use the notebook USGS_05470500_Daily_2001_2023.ipynb in GitHub repository: https://github.com/miguel-source/Water-Balance-Analysis-Tool
5. Compare **P − ET** vs **USGS runoff** via a scatter plot and linear regression.

> Tip: Update the **Parameters** cell with your own paths, watershed, and station IDs. List of watersheds for the state of Iowa can be found in this Zenodo: 



## 1) Environment & Requirements

Run the next cell once to install Python packages (skip if already installed in your environment).


In [ ]:

# If running in a fresh environment, uncomment to install dependencies.
# Note: In some managed/JupyterHub environments, use the terminal/conda instead.

# !pip install --upgrade pip
# !pip install geemap earthengine-api geopandas folium ipyleaflet shapely pyproj pillow scipy matplotlib pandas



## 2) Authenticate & Initialize Google Earth Engine

- The first time, you'll be prompted to sign in and paste an auth code.
- If you have a specific GCP **project ID** for Earth Engine, set it below.


In [ ]:

import ee

# Set your Earth Engine project (optional but recommended).
EE_PROJECT = 'ee-your-project-id'  # e.g., 'ee-mikediastat'  (Authors project - replace with your project if needed)

# Authenticate (first run will open a browser for auth)
try:
    ee.Initialize(project=EE_PROJECT)
except Exception:
    ee.Authenticate()
    ee.Initialize(project=EE_PROJECT)

print('Earth Engine initialized. Project:', EE_PROJECT)



## 3) Parameters

Update the watershed name, USGS station ID, file paths, date range, and export settings as needed.


In [ ]:

from pathlib import Path

# ---------------------- USER INPUTS ----------------------
# Watershed & station metadata
WATERSHED = "Ioway Creek"       # Folder name used in your local directory structure
USGS_STATION = "_05470500"      # USGS station code (matches your CSV file naming)

# Date range for analysis
START_YEAR = 2001
END_YEAR   = 2023               # inclusive

# Local paths (edit to your system) (This paths are from the Zenodo repository or the information pulled from StreamStats)
BASE_DIR = Path(r"C:/Users/adi10136/OneDrive - Iowa State University/Desktop/Runoff Analysis") # Change to your base directory
SHP_PATH = BASE_DIR / "Shp" / WATERSHED / "layers" / "globalwatershed.shp" # Path to your watershed shapefile
USGS_CSV = BASE_DIR / "Shp" / WATERSHED / f"{USGS_STATION}.csv" # Path to your USGS station CSV file
OUTPUT_DIR = BASE_DIR / "Outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Optional: Google Drive export folder name for GEE image exports
GDRIVE_FOLDER = "GEE_exports"

# Visualization for quick maps
VIS_PARAMS = {
    'min': 0,
    'max': 1000,
    'palette': ['#6A0D83', '#669DFF', '#C2FFF8', '#54FF19', '#FFF963', '#FF6F00', '#FF2200']
}

# Export scale (meters per pixel) for GEE image exports
EXPORT_SCALE = 4000  # Adjust to your needs (1000–5000 typical for CONUS-wide summaries)



## 4) Load Watershed Polygon & Convert to Earth Engine Geometry


In [ ]:

import geopandas as gpd

# Read watershed polygon
gdf = gpd.read_file(SHP_PATH)
# Ensure WGS84 for Earth Engine
gdf = gdf.to_crs(epsg=4326)

# Extract exterior ring coordinates (assumes a single-part polygon or first feature)
geom = gdf.geometry.iloc[0]
if geom.geom_type == 'MultiPolygon':
    geom = list(geom.geoms)[0]
coords = list(geom.exterior.coords)

# Create ee.Geometry.Polygon (expects a list-of-lists)
ee_polygon = ee.Geometry.Polygon([coords])

# Compute area in m^2 from a projected CRS for later hydrologic conversions
gdf_area = gdf.to_crs(epsg=5070)  # NAD83 / Conus Albers
total_area_m2 = float(gdf_area.geometry.area.sum())
print(f"Loaded watershed. Area = {total_area_m2:,.0f} m²")



## 5) Retrieve OpenET & GRIDMET, Compute Annual Sums and Watershed Means

- **OpenET**: `OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0`, band `et_ensemble_mad`
- **GRIDMET**: `IDAHO_EPSCOR/GRIDMET`, band `pr`
- We compute **annual sums** for each dataset, then **P − ET** and reduce by mean over the watershed polygon.


In [ ]:

import pandas as pd

years = list(range(START_YEAR, END_YEAR + 1))

# Collections
openet_ic = ee.ImageCollection('OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0')
gridmet_ic = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')

def annual_sum_imgcoll(ic, band, year):
    start = ee.Date.fromYMD(year, 1, 1)
    end   = ee.Date.fromYMD(year + 1, 1, 1)
    return ic.filterDate(start, end).select([band]).sum().set('year', year)

def region_mean(image, geom, scale):
    # Returns a dict with band-wise means
    return image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geom,
        scale=scale,
        bestEffort=True,
        maxPixels=1e13
    )

mean_p_minus_et = []
mean_p = []
mean_et = []

for yr in years:
    et_img = annual_sum_imgcoll(openet_ic, 'et_ensemble_mad', yr)
    p_img  = annual_sum_imgcoll(gridmet_ic, 'pr', yr)
    pet    = p_img.subtract(et_img).clip(ee_polygon)

    # Use a moderate scale (in meters) for polygon reduction; adjust as needed
    red_pet = region_mean(pet, ee_polygon, EXPORT_SCALE)
    red_p   = region_mean(p_img, ee_polygon, EXPORT_SCALE)
    red_et  = region_mean(et_img, ee_polygon, EXPORT_SCALE)

    # getInfo() calls fetch numbers to Python from EE servers
    pet_mean = red_pet.get('pr').getInfo() if red_pet.get('pr') is not None else None
    p_mean   = red_p.get('pr').getInfo()   if red_p.get('pr')   is not None else None
    et_mean  = red_et.get('et_ensemble_mad').getInfo() if red_et.get('et_ensemble_mad') is not None else None

    mean_p_minus_et.append(pet_mean)
    mean_p.append(p_mean)
    mean_et.append(et_mean)

df_pet = pd.DataFrame({
    'year': years,
    'P_mean': mean_p,           # units: mm (GRIDMET precipitation)
    'ET_mean': mean_et,         # units: mm (OpenET ET)
    'P_minus_ET_mean': mean_p_minus_et  # units: mm
}).set_index('year')

display(df_pet)



## 6) Quick Map (Optional)

Displays the **mean of annual ET sums** across your analysis period for context.
> Note: In static GitHub rendering you won't see the interactive map; run locally to view.


In [ ]:

import geemap

# Compute mean annual ET across period (2016–2021 in original; here use START_YEAR..END_YEAR as a demo)
def per_year_sum(y):
    start = ee.Date.fromYMD(y, 1, 1)
    end   = ee.Date.fromYMD(y + 1, 1, 1)
    return openet_ic.filterDate(start, end).select('et_ensemble_mad').sum().set('year', y)

years_list = ee.List.sequence(START_YEAR, END_YEAR)
yearly_sums_ic = ee.ImageCollection(years_list.map(per_year_sum))
mean_et_img = yearly_sums_ic.mean()

m = geemap.Map(center=[37.5, -95], zoom=4)
m.addLayer(mean_et_img, VIS_PARAMS, 'Mean of annual ET sums')
m.addLayer(ee.FeatureCollection(ee.Feature(ee_polygon)), {}, 'Watershed')
m.addLayerControl()
m



## 7) Optional: Export Annual (P − ET) Rasters to Google Drive

Set `EXPORT_SCALE`, `GDRIVE_FOLDER`, and choose years below. Exports are **started** as tasks—monitor in the GEE Tasks tab.


In [ ]:

EXPORT_YEARS = [END_YEAR]  # e.g., [2023] or list(range(START_YEAR, END_YEAR+1))

for yr in EXPORT_YEARS:
    start = ee.Date.fromYMD(yr, 1, 1)
    end   = ee.Date.fromYMD(yr + 1, 1, 1)
    et_img = openet_ic.filterDate(start, end).select('et_ensemble_mad').sum()
    p_img  = gridmet_ic.filterDate(start, end).select('pr').sum()
    runoff_img = p_img.subtract(et_img).clip(ee_polygon)

    task = ee.batch.Export.image.toDrive(
        image=runoff_img,
        description=f"Runoff_{yr}",
        folder=GDRIVE_FOLDER,
        fileNamePrefix=f"Runoff_{yr}",
        scale=EXPORT_SCALE,
        region=ee_polygon.bounds(),
        fileFormat="GeoTIFF",
        maxPixels=1e13
    )
    task.start()
    print(f"Export started for year {yr}. Check Earth Engine Tasks UI.")



## 8) USGS Discharge → Annual Runoff Depth (mm)

We read a CSV with daily (or finer) **Discharge** (in cfs), convert to m³/s, compute annual mean discharge, and convert to **runoff depth (mm)**:

\[ \text{Runoff depth (mm)} = \frac{\overline{Q}\;[m^3/s] \times \text{seconds/year}}{A\;[m^2]} \times 1000 \]


In [ ]:

import numpy as np

# Read USGS discharge CSV
usgs_df = pd.read_csv(USGS_CSV)

# Expect columns: 'Date' (YYYY-MM-DD), 'Discharge' (cfs). Adjust if your headers differ.
usgs_df['Date'] = pd.to_datetime(usgs_df['Date'])
usgs_df['Discharge_m3s'] = usgs_df['Discharge'] * 0.0283168  # cfs -> m^3/s

# Compute annual mean discharge (m^3/s)
usgs_annual = (
    usgs_df
    .assign(year=usgs_df['Date'].dt.year)
    .groupby('year', as_index=True)['Discharge_m3s']
    .mean()
    .rename('Q_mean_m3s')
    .to_frame()
)

SECONDS_PER_YEAR = 60 * 60 * 24 * 365  # ignore leap years for simplicity
usgs_annual['Runoff_mm'] = (usgs_annual['Q_mean_m3s'] * SECONDS_PER_YEAR / total_area_m2) * 1000.0

display(usgs_annual.head())
print(f"Computed runoff depth using watershed area = {total_area_m2:,.0f} m²")



## 9) Compare (P − ET) vs USGS Runoff

We join by **year** (inner join to keep only overlapping years), scatter-plot, and fit a linear regression.


In [ ]:

from scipy.stats import linregress
import matplotlib.pyplot as plt

# Prepare series aligned by year
pet_series = df_pet['P_minus_ET_mean'].dropna()
usgs_series = usgs_annual['Runoff_mm'].dropna()

# Join on overlapping years
df_join = pd.concat([pet_series, usgs_series], axis=1, join='inner').dropna()
df_join.columns = ['P_minus_ET_mm', 'USGS_Runoff_mm']

display(df_join.head())

x = df_join['USGS_Runoff_mm'].values
y = df_join['P_minus_ET_mm'].values

# Linear regression
slope, intercept, r_value, p_value, std_err = linregress(x, y)

plt.figure(figsize=(8, 6))
plt.scatter(x, y, label='Years')
plt.plot(x, slope * x + intercept, label=f'Fit: y = {slope:.2f}x + {intercept:.2f}', linewidth=2)
plt.xlabel('USGS Runoff (mm)')
plt.ylabel('P − ET (mm)')
plt.title('P − ET vs USGS Runoff')
plt.legend()
plt.grid(True)
png_path = OUTPUT_DIR / f"{WATERSHED}_{USGS_STATION}_PminusET_vs_USGS.png"
plt.savefig(png_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"R² = {r_value**2:.3f} | p = {p_value:.3g} | std_err = {std_err:.3g}")
print('Figure saved to:', png_path)



## 10) Notes & Troubleshooting

- **Complementary code**: Use complementary code to create the inputs or use the Zenodo for running the avaible watersheds in Iowa
- **Units**: GRIDMET `pr` is in **mm**, OpenET ET is **mm**. The difference (P − ET) is in **mm**.  
- **Watershed area**: Ensure the shapefile CRS used for area calculation is projected in meters (e.g., EPSG:26915/5070).  
- **Leap years**: The runoff depth conversion uses `365` days; if you want exact values, aggregate discharge to **annual volume** using actual day counts per year.  
- **GEE scale**: The `scale` parameter controls pixel resolution for `reduceRegion` and exports—coarser scales reduce compute & export time.  
- **GEE quotas**: Large polygons or small scales may hit `maxPixels` or timeouts. Consider increasing `scale`, or using `bestEffort=True`.  
- **CSV schema**: If your USGS CSV headers differ, adjust the column names in the code.
- **Maps on GitHub**: Interactive maps won’t render statically; run locally to view.

